In [8]:
apikey = "8cWjeJg2pd1YRVmbedEDy35Z70ELcu9qJD2qyzAbQLhD"

# Get an IAM token from IBM Cloud
url     = "https://iam.bluemix.net/oidc/token"
headers = { "Content-Type" : "application/x-www-form-urlencoded" }
data    = "apikey=" + apikey + "&grant_type=urn:ibm:params:oauth:grant-type:apikey"
IBM_cloud_IAM_uid = "bx"
IBM_cloud_IAM_pwd = "bx"
response  = requests.post( url, headers=headers, data=data, auth=( IBM_cloud_IAM_uid, IBM_cloud_IAM_pwd ) )
iam_token = response.json()["access_token"]

# from creditrisk.models import scorerisk
# p1 = scorerisk(CheckingStatus = "0_to_200", CreditHistory = "all_credits_paid_back", EmploymentDuration = "1_to_4", OwnsProperty = "real_estate", Age = 44, LoanAmount = 5000, LoanDuration = 24, CurrentResidenceDuration = 4, InstallmentPercent =3)
# p2 = scorerisk(CheckingStatus = "no_checking", CreditHistory = "prior_payments_delayed", EmploymentDuration = "4_to_7", OwnsProperty = "car_other", Age = 29, LoanAmount = 3000, LoanDuration = 18, CurrentResidenceDuration = 2, InstallmentPercent =5)

import urllib3, requests, json
ml_instance_id = "0c0a53ec-6937-47bc-b0a7-68840209659f"
# NOTE: generate iam_token and retrieve ml_instance_id based on provided documentation
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + iam_token, 'ML-Instance-ID': ml_instance_id}

In [11]:
import numpy as np
import requests

In [18]:
null = ""
# webobj = get_object_or_404(scorepayload)
payloadToScore = {"input_data":[{"fields":["CheckingStatus","LoanDuration","CreditHistory","LoanAmount","EmploymentDuration","InstallmentPercent","CurrentResidenceDuration","OwnsProperty","Age"],
                                     "values":[["less_0",24,"all_credits_paid_back",5000,"1_to_4",5,3,'real_estate',30]]}]}
score_respo = requests.post('https://us-south.ml.cloud.ibm.com/v4/deployments/3fd7ac36-5b32-43e5-b8f2-87c354deb094/predictions', json=payloadToScore, headers=header)


score_respo.text

'{\n  "trace": "02b592f497880d47900c42aee477d992",\n  "errors": [{\n    "code": "invalid_payload",\n    "message": "index 10 is out of bounds for axis 1 with size 9"\n  }]\n}'

In [ ]:

dictofvals = json.loads(score_respo.text).get('predictions')[0]
prediction = dictofvals['values'][0][0]
confidence = np.around((dictofvals['values'][0][1][0])*100, 2)
verdict = "empty"
if prediction == "No Risk":
    verdict = f"Our model has predicted that this customer is NOT at risk to repay this loan with {confidence}% confidence"
else:
    verdict = f"Our model has predicted that this customer is at risk to repay this loan  {confidence}% confidence"

print (verdict)